In [12]:
!pip install selenium


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [13]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from datetime import datetime

In [21]:
import pandas as pd
import re
import math

options = Options()
options.add_argument('--headless')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--window-size=1800,1200')
options.page_load_strategy = 'none'

driver = webdriver.Chrome(options=options)

error_list = []

years = range(2023, 2024)

for year in years:
    df_films = pd.DataFrame(columns=["Year", "Title", "Genres", "Length", "Letterboxd Ratings", "Directors"])
    num_films_collected = 0

    for n in range(0, 100):

        #72 films per page
        page = 1 + math.floor(n/72)
        num_on_page = n%72

        # resetting driver every 20 films. Driver seems to start glitching when it runs for too long. Runs faster after resetting.
        if((n%20 == 0) and (n!=0)):
            driver.close()
            driver.quit()
            time.sleep(2)
            driver = webdriver.Chrome(options=options)
        
        driver.get(f'https://letterboxd.com/films/popular/year/{year}/page/{page}')

        while True:
            try:
                film_element_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'poster-list')))
                film_elements = film_element_list.find_elements(By.TAG_NAME, 'li')
                driver.execute_script("arguments[0].scrollIntoView(true);", film_elements[num_on_page])
                film_elements[num_on_page].click()

                film_header = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'featured-film-header')))
            except Exception as e:
                print(type(e))
                print(f"\n Error when selecting film or getting film title. Resetting driver and trying again \n")
                error_list.append(f"\n Error when selecting film or getting film title. Resetting driver and trying again \n")

                driver.close()
                driver.quit()

                time.sleep(5)

                driver = webdriver.Chrome(options=options)
                driver.get(f'https://letterboxd.com/films/popular/year/{year}/page/{page}')
            else:
                film_title_element = film_header.find_element(By.TAG_NAME, "h1")
                film_title = film_title_element.text
                break

        ## film page. If film is longer than 300 mins or shorter than 75, skip it (it's a series or short film)

        try:
            text_footer_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "text-footer")))
        except Exception as e:
            print(type(e))
            print(f"\n Could not find film length for {film_title} \n")
            error_list.append(f"\n Could not find film length for {film_title} \n")
            film_length = None
        else:
            text_footer = text_footer_element.text
            match = re.search(r'\b([\d,]+)\b', text_footer)
            if match:
                film_length = int(match.group(0).replace(',', ''))
            else:
                print(f"\n Could not find film length for film {n} in {year} \n")
                film_length = None
        
        ## film page. Get total rating and each of the individual star ratings and save as a dict. 

        ratings = dict.fromkeys(["0.5", "1", "1.5", "2", "2.5", "3", "3.5", "4", "4.5", "5"])

        try:
            rating_histogram = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "rating-histogram")))
            rating_histogram_bars = driver.find_elements(By.CLASS_NAME, "rating-histogram-bar")
        except Exception as e:
            print(type(e))
            print(f"\n Could not find film ratings for {film_title} \n")
            error_list.append(f"\n Could not find film ratings for {film_title} \n")
            ratings = None
        else:
            for i in range(len(rating_histogram_bars)):

                # If 'a' element exists, text is stored in link "data-original-title", if not it is in parent element
                try:
                    link_element = rating_histogram_bars[i].find_element(By.TAG_NAME, "a")
                except Exception as e:
                    link_element = rating_histogram_bars[i]
            
                rating_text = link_element.get_attribute("data-original-title")
                if(i==0): ratings["0.5"] = rating_text
                if(i==1): ratings["1"] = rating_text
                if(i==2): ratings["1.5"] = rating_text
                if(i==3): ratings["2"] = rating_text
                if(i==4): ratings["2.5"] = rating_text
                if(i==5): ratings["3"] = rating_text
                if(i==6): ratings["3.5"] = rating_text
                if(i==7): ratings["4"] = rating_text
                if(i==8): ratings["4.5"] = rating_text
                if(i==9): ratings["5"] = rating_text

        ## film page. Scroll to Genres button and click to open genre tab. 
        genres = []

        try:
            genres_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-id="genres"]')))
            driver.execute_script("arguments[0].scrollIntoView(true);", genres_button)
            genres_button.click()
            genre_tab_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tab-genres")))
            genre_list_element = genre_tab_element.find_element(By.TAG_NAME, "p")
            genre_elements = genre_list_element.find_elements(By.TAG_NAME, "a")
        except Exception as e:
            print(type(e))
            print(f"\n Could not find genres for {film_title} \n")
            error_list.append(f"\n Could not find genres for {film_title} \n")
            genres = None
        else:
            for element in genre_elements:
                    genres.append(element.text)

        ## film page. Scroll to Crew button and click to open crew tab. 
        ## If no crew tab or director exists, skip this film. Some live performance recordings and things like that don't have a director or crew.
        ## Get director name. Click director name to go to director page. 
                    
        directors = []

        try:
            crew_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "crew")))
            driver.execute_script("arguments[0].scrollIntoView(true);", crew_button)
            crew_button.click()
            
            crew_tab_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tab-crew")))

            director_list_element = crew_tab_element.find_element(By.TAG_NAME, "p")
            director_list = director_list_element.find_elements(By.TAG_NAME, "a")

            crew_role_element = crew_tab_element.find_element(By.CLASS_NAME, "crewrole")
        except Exception as e:
            print(type(e))
            print(f"\n Could not find directors for {film_title} \n")
            error_list.append(f"\n Could not find directors for {film_title} \n")
            directors = None
        else:
            if("director" in crew_role_element.text.lower()):
                print(f"\n Could not find directors for {film_title} \n")
                directors = None
            else:
                main_window = driver.current_window_handle

                for i in range(len(director_list)):

                    ## Getting these elements again to avoid stale element exception
                    crew_tab_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tab-crew")))
                    director_list_element = crew_tab_element.find_element(By.TAG_NAME, "p")
                    director_list = director_list_element.find_elements(By.TAG_NAME, "a")

                    driver.execute_script("arguments[0].scrollIntoView(true);", director_list[i])
                    name = director_list[i].text
                    
                    director_list[i].send_keys(Keys.COMMAND + Keys.RETURN)
                    new_tab = WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    driver.switch_to.window(driver.window_handles[1])

                    ## Director page. Click TMDB element to go to TMDB director page.

                    try: 
                        sidebar_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "sidebar")))
                        TMDB_element = sidebar_element.find_element(By.LINK_TEXT, "TMDB")
                    except Exception as e:
                        print(type(e))
                        print(f"No Tmdb page for director {name} in film {film_title}")
                        error_list.append(f"No Tmdb page for director {name} in film {film_title}")

                        gender = None
                        birthdate = None
                    else:
                        driver.execute_script("arguments[0].scrollIntoView(true);", TMDB_element)

                        # Somtimes clicking TMDB button doesn't work, so we will loop clicking it until the driver url shows it has left letterboxd page
                        z = 0
                        while True:
                            if("letterboxd" in driver.current_url):
                                TMDB_element.click()
                            else:
                                break

                            if z > 0:
                                time.sleep(2)

                            z+=1

                        ## TMDB page. Sometimes not loading elements that should be there, so we have an endless loop reloading page if searching for elements throws exception
                        # Facts element is definitely there for every director page even if it doesn't contain gender and birthdate info
                        # Should set gender and birthdate to "-" if the info is not there
                        
                        for y in range(5):
                            try: 
                                facts_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'facts')))
                                facts = facts_element.find_elements(By.TAG_NAME, "p")
                            except Exception as e:
                                print(type(e))
                                print(f"Couldn't find facts element on {name} TMDB page for film {film_title}. Refreshing page and trying again")
                                #error_list.append(f"Couldn't find facts element on {name} TMDB page for film {film_title}. Refreshing page and trying again")
                                url = driver.current_url
                                driver.get(url)
                                time.sleep(5)

                                if y == 4:
                                    print(f"No gender and birthdate info found on TMDB page for {name} for film {film_title} after refreshing 5 times")
                                    error_list.append(f"No gender and birthdate info found on TMDB page for {name} for film {film_title} after refreshing 5 times")

                                gender = None
                                birthdate = None
                            else:
                                gender = facts[2].text
                                birthdate = facts[3].text
                                break

                    directors.append([name, gender, birthdate])
                    
                    driver.close()
                    new_tab = WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(1))
                    driver.switch_to.window(driver.window_handles[0])
                    



        new_row = pd.DataFrame({"Year":[year], 
                                "Title":[film_title], 
                                "Genres":[genres], 
                                "Length":[film_length], 
                                "Letterboxd Ratings":[ratings], 
                                "Directors":[directors]})
        
        df_films = pd.concat([df_films, new_row], ignore_index=True)
        num_films_collected += 1
        print(f"{film_title} added to dataframe")
        print(f"{num_films_collected} films in {year} added to dataframe")

    df_films.to_csv(f"film-data/films-{year}.csv")

driver.close()
driver.quit()


<class 'selenium.common.exceptions.TimeoutException'>
Couldn't find facts element on Greta Gerwig TMDB page for film Barbie. Refreshing page and trying again
Barbie added to dataframe
1 films in 2023 added to dataframe
Oppenheimer added to dataframe
2 films in 2023 added to dataframe
Spider-Man: Across the Spider-Verse added to dataframe
3 films in 2023 added to dataframe
Saltburn added to dataframe
4 films in 2023 added to dataframe
Poor Things added to dataframe
5 films in 2023 added to dataframe
Guardians of the Galaxy Vol. 3 added to dataframe
6 films in 2023 added to dataframe
Past Lives added to dataframe
7 films in 2023 added to dataframe
Bottoms added to dataframe
8 films in 2023 added to dataframe
Killers of the Flower Moon added to dataframe
9 films in 2023 added to dataframe
The Super Mario Bros. Movie added to dataframe
10 films in 2023 added to dataframe
Asteroid City added to dataframe
11 films in 2023 added to dataframe
The Hunger Games: The Ballad of Songbirds & Snakes 

In [22]:
error_list

['\n Error when selecting film or getting film title. Resetting driver and trying again \n',
 '\n Error when selecting film or getting film title. Resetting driver and trying again \n']

In [ ]:
df_films

,Year,Title,Genres,Length,Letterboxd Ratings,Directors
0,1994,Pulp Fiction,"[Crime, Thriller]",154,"{'0.5': '4,033 half-★ ratings (0%)', '1': '10,...","[[Quentin Tarantino, Gender\nMale, Birthday\nM..."
1,1994,Forrest Gump,"[Drama, Comedy, Romance]",142,"{'0.5': '4,734 half-★ ratings (0%)', '1': '11,...","[[Robert Zemeckis, Gender\nMale, Birthday\nMay..."
2,1994,The Shawshank Redemption,"[Crime, Drama]",142,"{'0.5': '931 half-★ ratings (0%)', '1': '1,775...","[[Frank Darabont, Gender\nMale, Birthday\nJanu..."
3,1994,The Lion King,"[Animation, Family, Drama]",89,"{'0.5': '1,330 half-★ ratings (0%)', '1': '3,5...","[[Roger Allers, Gender\nMale, Birthday\nJune 2..."
4,1994,Léon: The Professional,"[Crime, Action, Drama]",111,"{'0.5': '3,067 half-★ ratings (0%)', '1': '4,6...","[[Luc Besson, Gender\nMale, Birthday\nMarch 18..."
5,1994,Chungking Express,"[Comedy, Drama, Romance]",103,"{'0.5': '391 half-★ ratings (0%)', '1': '677 ★...","[[Wong Kar-wai, Gender\nMale, Birthday\nJuly 1..."
6,1994,The Mask,"[Fantasy, Romance, Crime, Comedy]",101,"{'0.5': '2,535 half-★ ratings (1%)', '1': '7,2...","[[Chuck Russell, Gender\nMale, Birthday\nMay 9..."
7,1994,Interview with the Vampire,"[Horror, Drama, Fantasy]",123,"{'0.5': '763 half-★ ratings (0%)', '1': '2,302...","[[Neil Jordan, Gender\nMale, Birthday\nFebruar..."
8,1994,Speed,"[Adventure, Action, Crime]",116,"{'0.5': '311 half-★ ratings (0%)', '1': '1,226...","[[Jan de Bont, Gender\nMale, Birthday\nOctober..."
9,1994,Natural Born Killers,"[Thriller, Drama, Crime]",118,"{'0.5': '1,463 half-★ ratings (1%)', '1': '3,0...","[[Oliver Stone, Gender\nMale, Birthday\nSeptem..."


In [ ]:
df_films.to_csv(f"film-data/films-{year}.csv")